In [10]:
import numpy as np
import random
import math
from scipy.ndimage import map_coordinates
from scipy.ndimage import rotate
from scipy.ndimage import shift
import matplotlib.pyplot as plt
from os import listdir
import PIL
from PIL import Image
import cv2



training_label_folders = ['/Users/apple/Sync/Research/NiCr XPEEM/Labeled Images/Movie 3 random crops/Labels',
            '/Users/apple/Sync/Research/NiCr XPEEM/Labeled Images/Movie 2 random crops/Labels',
            '/Users/apple/Sync/Research/NiCr XPEEM/Labeled Images/Time Series mov3 location 256_320/Labels',
            '/Users/apple/Sync/Research/NiCr XPEEM/Labeled Images/Spatial Mov3 Frame87/Labels']

training_image_folders = ['/Users/apple/Sync/Research/NiCr XPEEM/Labeled Images/Movie 3 random crops/Images',
            '/Users/apple/Sync/Research/NiCr XPEEM/Labeled Images/Movie 2 random crops/Images',
            '/Users/apple/Sync/Research/NiCr XPEEM/Labeled Images/Time Series mov3 location 256_320/Images',
            '/Users/apple/Sync/Research/NiCr XPEEM/Labeled Images/Spatial Mov3 Frame87/Images']


Images need to be normalized from 0 to 1, relative to the overal intensity of the 1024x1024 image not the overall intensity of an individual patch

In [11]:
from os import listdir
import PIL
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

def normalize_folder(datafolder,outfolder):
    """Takes all of the .tif files in a folder and divides them by a constant to normalize them"""
    foldername = datafolder
    image_names = listdir(foldername)

    for image_name in image_names:
        if ".tif" not in image_name:
            continue
        print(image_name)

        image = plt.imread(foldername+image_name)
        image= image/65530

        savecrop = Image.fromarray(image)
        imageaddress = outfolder+image_name
        savecrop.save(imageaddress)
    

    return



def tif_to_npy(datafolder,outfolder):
    """Converts all the .tif files in a folder to .npy files """
    foldername = datafolder
    image_names = listdir(foldername)

    for image_name in image_names:
        if ".tif" not in image_name:
            continue
        print(image_name)

        image = plt.imread(foldername+image_name)
        imageaddress = outfolder+image_name
        imageaddress = imageaddress[:-4]
        np.save(imageaddress,image)
    return


def image_label_zipper(labelfolder,imagefolder,outfolder):
    image_names = listdir(imagefolder)
    label_names = listdir(labelfolder)
    counter = 0
    for label_name in label_names:
        if '.tif' not in label_name:
            continue


        img = plt.imread(imagefolder+label_name)
        lbl = plt.imread(labelfolder+label_name)

        imageaddress = outfolder+"label_"+str(counter)
        np.savez(imageaddress,image=img, label=lbl)
        counter+=1


    return

In [12]:
#tif_to_npy('/Users/apple/Sync/Research/NiCr XPEEM/Labeled Images/png labels/','/Users/apple/Sync/Research/NiCr XPEEM/Labeled Images/npy labels/')

image_label_zipper('/Users/apple/Sync/Research/NiCr XPEEM/Labeled Images/png labels/','/Users/apple/Sync/Research/NiCr XPEEM/Labeled Images/png images/', '/Users/apple/Sync/Research/NiCr XPEEM/Labeled Images/zipped npy labels/')

test = np.load('/Users/apple/Sync/Research/NiCr XPEEM/Labeled Images/zipped npy labels/label_38.npz')

print(test['label'].shape)


(64, 64)


In [15]:
from scipy import ndimage

def random_rot_flip(image, label):
    k = np.random.randint(0, 4)
    image = np.rot90(image, k)
    label = np.rot90(label, k)
    axis = np.random.randint(0, 2)
    image = np.flip(image, axis=axis).copy()
    label = np.flip(label, axis=axis).copy()
    return image, label

def zip_and_augment(labelfolder, imagefolder,outfolder):
    image_names = listdir(imagefolder)
    label_names = listdir(labelfolder)
    counter = 0
    for label_name in label_names:
        if '.npz' not in label_name:
            continue


        imglabel = np.load(imagefolder+label_name)
        img = imglabel["image"]
        lbl = imglabel["label"]
        img, lbl = random_rot_flip(img,lbl)

        imageaddress = outfolder+"augmented_"+str(counter)
        np.savez(imageaddress,image=img, label=lbl)
        counter+=1

        img, lbl = random_rot_flip(img,lbl)
        imageaddress = outfolder+"augmented_"+str(counter)
        np.savez(imageaddress,image=img, label=lbl)
        counter+=1


    return

zip_and_augment('/Users/apple/MLIAFinalProject/Data/npzs/','/Users/apple/MLIAFinalProject/Data/npzs/', '/Users/apple/Sync/Research/NiCr XPEEM/Labeled Images/zipped npy labels/')

test = np.load('/Users/apple/Sync/Research/NiCr XPEEM/Labeled Images/zipped npy labels/augmented_38.npz')

print(test['label'].shape)



In [16]:
zip_and_augment('/Users/apple/MLIAFinalProject/Data/npzs/','/Users/apple/MLIAFinalProject/Data/npzs/', '/Users/apple/Sync/Research/NiCr XPEEM/Labeled Images/zipped npy labels/')

test = np.load('/Users/apple/Sync/Research/NiCr XPEEM/Labeled Images/zipped npy labels/augmented_38.npz')

print(test['label'].shape)

(64, 64)
